# Fine-tuning : RF-DETR

Objectif : faire de RF-DETR un détecteur de poissons sur des images de 1920\*1080 pixels

In [1]:
# Importations
import os
import numpy as np
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import json
import cv2
import matplotlib.pyplot as plt
import pycocotools.coco as coco
from PIL import Image
from rfdetr import RFDETRLarge, RFDETRBase  # Assurez-vous que le package rf-detr est bien installé


# Vérification de la disponibilité du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device utilisé :", device)


Device utilisé : cuda


In [2]:
data_dir = "/home/mateo/Travail/Fine-tune/Dataset/Roboflow"  # chemin vers votre jeu de données
#images_dir = os.path.join(data_dir, "luderick-seagrass/train")
#annotations_file = os.path.join(images_dir, "_annotations.coco.json")

# Charger les annotations COCO
#coco_dataset = coco.COCO(annotations_file)
#print("Nombre d'images dans le dataset :", len(coco_dataset.imgs))


In [3]:
# Cellule 6 : Charger et adapter le modèle rf-detr pré-entraîné
# -----------------------------------------------------
# Nous allons charger le modèle rf-detr du dépôt et le fine-tuner
# Vous pouvez soit utiliser le modèle préentraîné fourni par rf-detr (si disponible) ou initialiser un modèle de DETR et le charger
# Vous pouvez adapter le nombre de classes. Ici, nous avons 1 classe (poisson) + 1 classe pour le fond si nécessaire.
num_classes = 2  # par exemple, 1 classe pour "poisson" et 1 classe pour le "fond"

# Chargement du modèle pré-entraîné (précisez éventuellement le checkpoint préentraîné)
model = RFDETRBase(num_classes=num_classes, pretrained=True)
#model.to(device)
print("Modèle chargé sur l'appareil :", device)


num_classes mismatch: pretrain weights has 90 classes, but your model has 2 classes
reinitializing detection head with 90 classes


Loading pretrain weights
Modèle chargé sur l'appareil : cuda


In [ ]:
model.train(dataset_dir=os.path.join(data_dir, "Fish.v44i.coco"), epochs=3, batch_size=1, grad_accum_steps=1, lr=1e-4)
Image.open("/content/output/metrics_plot.png")

num_classes mismatch: model has 2 classes, but your dataset has 14 classes
reinitializing your detection head with 14 classes.


TensorBoard logging initialized. To monitor logs, use 'tensorboard --logdir output' and open http://localhost:6006/ in browser.
Not using distributed mode
git:
  sha: N/A, status: clean, branch: N/A

Namespace(num_classes=14, grad_accum_steps=1, amp=True, lr=0.0001, lr_encoder=0.00015, batch_size=1, weight_decay=0.0001, epochs=3, lr_drop=100, clip_max_norm=0.1, lr_vit_layer_decay=0.8, lr_component_decay=0.7, do_benchmark=False, dropout=0, drop_path=0.0, drop_mode='standard', drop_schedule='constant', cutoff_epoch=0, pretrained_encoder=None, pretrain_weights='rf-detr-base.pth', pretrain_exclude_keys=None, pretrain_keys_modify_to_load=None, pretrained_distiller=None, encoder='dinov2_windowed_small', vit_encoder_num_layers=12, window_block_indexes=None, position_embedding='sine', out_feature_indexes=[2, 5, 8, 11], freeze_encoder=False, layer_norm=True, rms_norm=False, backbone_lora=False, force_no_pretrain=False, dec_layers=3, dim_feedforward=2048, hidden_dim=256, sa_nheads=8, ca_nheads=1

fatal: not a git repository (or any of the parent directories): .git


Get benchmark
Start training
Grad accum steps:  1
Total batch size:  1
LENGTH OF DATA LOADER: 476


Epoch: [0]  [  0/476]  eta: 0:08:50  lr: 0.000100  class_error: 92.31  loss: 7.8890 (7.8890)  loss_ce: 1.3693 (1.3693)  loss_bbox: 0.2988 (0.2988)  loss_giou: 0.1779 (0.1779)  loss_ce_0: 1.3657 (1.3657)  loss_bbox_0: 0.3639 (0.3639)  loss_giou_0: 0.2148 (0.2148)  loss_ce_1: 1.3626 (1.3626)  loss_bbox_1: 0.3751 (0.3751)  loss_giou_1: 0.2075 (0.2075)  loss_ce_enc: 1.3058 (1.3058)  loss_bbox_enc: 0.5617 (0.5617)  loss_giou_enc: 0.2860 (0.2860)  loss_ce_unscaled: 1.3693 (1.3693)  class_error_unscaled: 92.3077 (92.3077)  loss_bbox_unscaled: 0.0598 (0.0598)  loss_giou_unscaled: 0.0889 (0.0889)  cardinality_error_unscaled: 3890.0000 (3890.0000)  loss_ce_0_unscaled: 1.3657 (1.3657)  loss_bbox_0_unscaled: 0.0728 (0.0728)  loss_giou_0_unscaled: 0.1074 (0.1074)  cardinality_error_0_unscaled: 3615.0000 (3615.0000)  loss_ce_1_unscaled: 1.3626 (1.3626)  loss_bbox_1_unscaled: 0.0750 (0.0750)  loss_giou_1_unscaled: 0.1038 (0.1038)  cardinality_error_1_unscaled: 2797.0000 (2797.0000)  loss_ce_enc_unsca

Epoch: [0]  [ 70/476]  eta: 0:04:23  lr: 0.000100  class_error: 82.05  loss: 9.5720 (9.7811)  loss_ce: 1.3155 (1.2647)  loss_bbox: 0.5681 (0.5954)  loss_giou: 0.3219 (0.3941)  loss_ce_0: 1.2628 (1.2174)  loss_bbox_0: 0.6203 (0.7606)  loss_giou_0: 0.3764 (0.4597)  loss_ce_1: 1.2864 (1.2540)  loss_bbox_1: 0.5809 (0.6424)  loss_giou_1: 0.3692 (0.4151)  loss_ce_enc: 1.1396 (1.1354)  loss_bbox_enc: 0.8627 (1.0297)  loss_giou_enc: 0.5228 (0.6126)  loss_ce_unscaled: 1.3155 (1.2647)  class_error_unscaled: 100.0000 (88.3153)  loss_bbox_unscaled: 0.1136 (0.1191)  loss_giou_unscaled: 0.1610 (0.1970)  cardinality_error_unscaled: 3899.0000 (3894.4085)  loss_ce_0_unscaled: 1.2628 (1.2174)  loss_bbox_0_unscaled: 0.1241 (0.1521)  loss_giou_0_unscaled: 0.1882 (0.2299)  cardinality_error_0_unscaled: 3896.0000 (3818.4085)  loss_ce_1_unscaled: 1.2864 (1.2540)  loss_bbox_1_unscaled: 0.1162 (0.1285)  loss_giou_1_unscaled: 0.1846 (0.2075)  cardinality_error_1_unscaled: 3889.0000 (3601.5493)  loss_ce_enc_unsc

Epoch: [0]  [140/476]  eta: 0:03:38  lr: 0.000100  class_error: 59.62  loss: 9.2188 (9.7121)  loss_ce: 1.2670 (1.2712)  loss_bbox: 0.4516 (0.5968)  loss_giou: 0.4039 (0.4019)  loss_ce_0: 1.2344 (1.2315)  loss_bbox_0: 0.5431 (0.6999)  loss_giou_0: 0.4375 (0.4502)  loss_ce_1: 1.2339 (1.2614)  loss_bbox_1: 0.4696 (0.6296)  loss_giou_1: 0.4261 (0.4173)  loss_ce_enc: 1.1312 (1.1360)  loss_bbox_enc: 0.7997 (0.9934)  loss_giou_enc: 0.5399 (0.6228)  loss_ce_unscaled: 1.2670 (1.2712)  class_error_unscaled: 69.2308 (82.6356)  loss_bbox_unscaled: 0.0903 (0.1194)  loss_giou_unscaled: 0.2019 (0.2010)  cardinality_error_unscaled: 3899.0000 (3896.0993)  loss_ce_0_unscaled: 1.2344 (1.2315)  loss_bbox_0_unscaled: 0.1086 (0.1400)  loss_giou_0_unscaled: 0.2188 (0.2251)  cardinality_error_0_unscaled: 3893.0000 (3855.2908)  loss_ce_1_unscaled: 1.2339 (1.2614)  loss_bbox_1_unscaled: 0.0939 (0.1259)  loss_giou_1_unscaled: 0.2131 (0.2087)  cardinality_error_1_unscaled: 3891.0000 (3742.8865)  loss_ce_enc_unsca

Epoch: [0]  [210/476]  eta: 0:02:54  lr: 0.000100  class_error: 38.46  loss: 9.1808 (9.7554)  loss_ce: 1.3478 (1.2798)  loss_bbox: 0.4672 (0.5996)  loss_giou: 0.2609 (0.4120)  loss_ce_0: 1.3522 (1.2370)  loss_bbox_0: 0.5571 (0.6855)  loss_giou_0: 0.2813 (0.4544)  loss_ce_1: 1.3674 (1.2644)  loss_bbox_1: 0.5107 (0.6410)  loss_giou_1: 0.2601 (0.4298)  loss_ce_enc: 1.1931 (1.1399)  loss_bbox_enc: 0.8973 (0.9879)  loss_giou_enc: 0.4950 (0.6242)  loss_ce_unscaled: 1.3478 (1.2798)  class_error_unscaled: 84.6154 (82.0567)  loss_bbox_unscaled: 0.0934 (0.1199)  loss_giou_unscaled: 0.1304 (0.2060)  cardinality_error_unscaled: 3899.0000 (3896.7346)  loss_ce_0_unscaled: 1.3522 (1.2370)  loss_bbox_0_unscaled: 0.1114 (0.1371)  loss_giou_0_unscaled: 0.1406 (0.2272)  cardinality_error_0_unscaled: 3899.0000 (3868.7156)  loss_ce_1_unscaled: 1.3674 (1.2644)  loss_bbox_1_unscaled: 0.1021 (0.1282)  loss_giou_1_unscaled: 0.1301 (0.2149)  cardinality_error_1_unscaled: 3899.0000 (3792.1659)  loss_ce_enc_unsca

In [ ]:
# Crée une instance du modèle
model = RFDETRBase()
# Affiche un résumé rapide du modèle
print(model)